In [28]:
import os
import pandas as pd
import datetime as dt
import spotipy
from secrets import secrets
from sqlalchemy import create_engine
from spotipy.oauth2 import SpotifyOAuth

In [2]:
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=secrets["SPOTIFY_CLIENT_ID"],
                                               client_secret=secrets["SPOTIFY_CLIENT_SECRET"],
                                               redirect_uri="http://localhost:8892/callback",
                                               scope="user-read-recently-played user-top-read"))

In [29]:
RANGES = {'short_term': 'Short', 'medium_term': 'Medium', 'long_term': 'Long'}
LIMIT = 50

def get_user_df(sp):
    user = sp.me()
    df_user = pd.DataFrame({
        'user_id': user['id'],
        'display_name': user['display_name'],
        'spotify_url': user['external_urls']['spotify'],
        'image_url': user['images'][0]['url'],
        'followers': user['followers']['total'],
        'date_created': dt.datetime.now(),
        'last_login': dt.datetime.now()
    }, index=[0])
    return df_user

def get_recently_played_df(sp):
    user_id = sp.me()['id']
    recently_played = sp.current_user_recently_played()
    dict_list = []
    for rp in recently_played['items']:
        this_track = {
            'user_id': user_id,
            'track': rp['track']['name'],
            'artists': "; ".join(a['name'] for a in rp['track']['artists']),
            'album': rp['track']['album']['name'],
            'release_date': rp['track']['album']['release_date'],
            'track_url': rp['track']['external_urls']['spotify'],
            'played_at': rp['played_at']
        }
        dict_list.append(this_track)
    return pd.DataFrame.from_dict(dict_list)

def get_current_playlists_df(sp):
    user_id = sp.me()['id']
    user_playlists = sp.current_user_playlists()
    dict_list = []
    for pl in user_playlists['items']:
        this_playlist = {
            'user_id': user_id,
            'playlist_id': pl['id'],
            'name': pl['name'],
            'description': pl['description'],
            'owner_display_name': pl['owner']['display_name'],
            'playlist_url': pl['external_urls']['spotify'],
            'track_numbers': pl['tracks']['total']
        }
        dict_list.append(this_playlist)
    return pd.DataFrame.from_dict(dict_list)

def get_top_artists_df(sp):
    user_id = sp.me()['id']
    dict_list = []
    for r in RANGES:
        top_artists = sp.current_user_top_artists(time_range=r, limit=LIMIT)
        for i, a in enumerate(top_artists['items']):
            this_artist = {
                'user_id': user_id,
                'rank': i+1,
                'artist': a['name'],
                'genres': "; ".join(g for g in a['genres']),
                'artist_url': a['external_urls']['spotify'],
                'timeframe': RANGES[r]
            }
            dict_list.append(this_artist)
    return pd.DataFrame.from_dict(dict_list)

def get_top_tracks_df(sp):
    user_id = sp.me()['id']
    dict_list = []
    for r in RANGES:
        top_tracks = sp.current_user_top_tracks(time_range=r, limit=LIMIT)
        for i, t in enumerate(top_tracks['items']):
            this_track = {
                'user_id': user_id,
                'rank': i+1,
                'track': t['name'],
                'artists': "; ".join(a['name'] for a in t['artists']),
                'album': t['album']['name'],
                'release_date': t['album']['release_date'],
                'track_url': t['external_urls']['spotify'],
                'timeframe': RANGES[r]
            }
            dict_list.append(this_track)
    return pd.DataFrame.from_dict(dict_list)

In [4]:
df_user = get_user_df(sp)
df_user.head()

,user_id,display_name,spotify_url,image_url,followers,date_created,last_login
0,12120382831,Bin Xuan Kong,https://open.spotify.com/user/12120382831,https://scontent-hkt1-2.xx.fbcdn.net/v/t1.0-1/...,53,2020-12-27 18:24:31.543185,2020-12-27 18:24:31.543192


In [5]:
df_rp = get_recently_played_df(sp)
df_rp.head()

,user_id,track,artists,album,release_date,track_url,played_at
0,12120382831,Until You Were Gone,The Chainsmokers; Tritonal; Emily Warren,Until You Were Gone,2015-09-18,https://open.spotify.com/track/12MqM3S0fTvR6X4...,2020-12-27T08:42:19.389Z
1,12120382831,Old Thing Back (feat. Ja Rule and Ralph Tresvant),Matoma; The Notorious B.I.G.; Ja Rule; Ralph T...,Old Thing Back (feat. Ja Rule and Ralph Tresvant),2015-02-17,https://open.spotify.com/track/6HTJZ0TQJVMSKkU...,2020-12-27T08:38:43.404Z
2,12120382831,Surrender,Cash Cash,"Blood, Sweat & 3 Years",2016-06-24,https://open.spotify.com/track/5NiTFfCUp0UrjvS...,2020-12-27T08:33:20.267Z
3,12120382831,Disarm You (feat. Ilsey),Kaskade; Ilsey,Automatic,2015-09-25,https://open.spotify.com/track/3xQjivjUZ4KnUsl...,2020-12-27T08:28:50.162Z
4,12120382831,Disarm You (feat. Ilsey),Kaskade; Ilsey,Automatic,2015-09-25,https://open.spotify.com/track/3xQjivjUZ4KnUsl...,2020-12-27T06:39:55.847Z


In [6]:
df_cp = get_current_playlists_df(sp)
df_cp.head()

,user_id,playlist_id,name,description,owner_display_name,playlist_url,track_numbers
0,12120382831,3EE6g5tgJxg4dh1iLBjszN,rewind,for the memories,Bin Xuan Kong,https://open.spotify.com/playlist/3EE6g5tgJxg4...,45
1,12120382831,1yiei0loSfJ5Dw6V8FX3HK,gang gang gang,for the turnt,Bin Xuan Kong,https://open.spotify.com/playlist/1yiei0loSfJ5...,44
2,12120382831,1lCAKTr58quQSN3kG3rVVc,good vibes only,for the mood,Bin Xuan Kong,https://open.spotify.com/playlist/1lCAKTr58quQ...,46
3,12120382831,1tQnCB9tYc43xuiC7i6odH,s a d b o y s,for the nights,Bin Xuan Kong,https://open.spotify.com/playlist/1tQnCB9tYc43...,46
4,12120382831,7c0J78KSxxDrADgG0BpJU9,feelin blue,for the sad,Bin Xuan Kong,https://open.spotify.com/playlist/7c0J78KSxxDr...,42


In [7]:
df_ta = get_top_artists_df(sp)
df_ta.head()

,user_id,rank,artist,genres,artist_url,timeframe
0,12120382831,1,Jay Chou,c-pop; mandopop; taiwan pop; zhongguo feng,https://open.spotify.com/artist/2elBjNSdBE2Y3f...,Short
1,12120382831,2,Kid Cudi,ohio hip hop; pop rap; rap,https://open.spotify.com/artist/0fA0VVWsXO9YnA...,Short
2,12120382831,3,Joji,alternative r&b; viral pop,https://open.spotify.com/artist/3MZsBdqDrRTJih...,Short
3,12120382831,4,Calvin Harris,dance pop; edm; electro house; house; pop; pop...,https://open.spotify.com/artist/7CajNmpbOovFoO...,Short
4,12120382831,5,Kanye West,chicago rap; rap,https://open.spotify.com/artist/5K4W6rqBFWDnAN...,Short


In [8]:
df_tt = get_top_tracks_df(sp)
df_tt.head()

,user_id,rank,track,artists,album,release_date,track_url,timeframe
0,12120382831,1,安靜,Jay Chou,范特西,2001-09-14,https://open.spotify.com/track/4FOydQ5IdqmdeBg...,Short
1,12120382831,2,With You (Jai Wolf Remix),Dirty South; FMLYBND,With You (The Remixes),2015-04-28,https://open.spotify.com/track/5H2cgEPEJxA72Bq...,Short
2,12120382831,3,Nikes,Frank Ocean,Blonde,2016-08-20,https://open.spotify.com/track/19YKaevk2bce4od...,Short
3,12120382831,4,算什麼男人,Jay Chou,哎呦 不錯哦,2014-12-26,https://open.spotify.com/track/7vs3JM90lKhp11a...,Short
4,12120382831,5,千里之外,Jay Chou,依然范特西,2006-09-05,https://open.spotify.com/track/02K5L2D21TVIINi...,Short


In [21]:
# TO SQL
engine = create_engine(secrets['DATABASE_URL'])
df_user.to_sql('Users', engine, index=False, if_exists='replace')
df_rp.to_sql('RecentlyPlayed', engine, index=False, if_exists='replace')
df_cp.to_sql('CurrentPlaylists', engine, index=False, if_exists='replace')
df_ta.to_sql('TopArtists', engine, index=False, if_exists='replace')
df_tt.to_sql('TopTracks', engine, index=False, if_exists='replace')

In [9]:
user_profile = df_user.to_dict('records')[0]
user_profile

{'user_id': '12120382831',
 'display_name': 'Bin Xuan Kong',
 'spotify_url': 'https://open.spotify.com/user/12120382831',
 'image_url': 'https://scontent-hkt1-2.xx.fbcdn.net/v/t1.0-1/p320x320/11988649_10205375733654944_669349554023656758_n.jpg?_nc_cat=110&ccb=2&_nc_sid=0c64ff&_nc_ohc=AJow9AVGs5YAX8M8_c_&_nc_ht=scontent-hkt1-2.xx&tp=6&oh=7e0addad9882e303c4df5928dd93401f&oe=600D9BD2',
 'followers': 53,
 'date_created': Timestamp('2020-12-27 18:24:31.543185'),
 'last_login': Timestamp('2020-12-27 18:24:31.543192')}

In [37]:
DATABASE_URL = os.environ.get('DATABASE_URL')

user_update_query = """
UPDATE "Users"
SET display_name = %(display_name)s, spotify_url = %(spotify_url)s, image_url = %(image_url)s,
    followers = %(followers)s, last_login = %(last_login)s
WHERE user_id = %(user_id)s
"""

def update_user_profile(df_user):
    engine = create_engine(DATABASE_URL)
    u = df_user.to_dict('records')[0]
    df_exist = pd.read_sql_query('SELECT * FROM "Users" u WHERE u.user_id = %(user_id)s', engine, \
                                 params={'user_id': u['user_id']})
    if len(df_exist) == 0:
        df_user.to_sql('Users', engine, if_exists='append', index=False)
    else:
        engine.execute(user_update_query, user_id=u['user_id'], display_name=u['display_name'], spotify_url=u['spotify_url'], \
            image_url=u['image_url'], followers=u['followers'], last_login=u['last_login'])
    engine.dispose()
    return u

In [27]:
update_user_profile(df_user)

{'user_id': '12120382831',
 'display_name': 'Bin Xuan Kong',
 'spotify_url': 'https://open.spotify.com/user/12120382831',
 'image_url': 'https://scontent-hkt1-2.xx.fbcdn.net/v/t1.0-1/p320x320/11988649_10205375733654944_669349554023656758_n.jpg?_nc_cat=110&ccb=2&_nc_sid=0c64ff&_nc_ohc=AJow9AVGs5YAX8M8_c_&_nc_ht=scontent-hkt1-2.xx&tp=6&oh=7e0addad9882e303c4df5928dd93401f&oe=600D9BD2',
 'followers': 53,
 'date_created': Timestamp('2020-12-27 18:24:31.543185'),
 'last_login': Timestamp('2020-12-27 18:24:31.543192')}

In [30]:
df2 = pd.DataFrame({'user_id': '1', 'display_name': 'Test', 'spotify_url': 'x', 'image_url': 'y',
                    'followers': 10, 'date_created': dt.datetime.now(), 'last_login': dt.datetime.now()}, index=[0])
df2

,user_id,display_name,spotify_url,image_url,followers,date_created,last_login
0,1,Test,x,y,10,2020-12-27 18:41:19.184128,2020-12-27 18:41:19.184134


In [38]:
update_user_profile(df2)

{'user_id': '1',
 'display_name': 'Test',
 'spotify_url': 'x',
 'image_url': 'y',
 'followers': 10,
 'date_created': Timestamp('2020-12-27 18:41:19.184128'),
 'last_login': Timestamp('2020-12-27 18:41:19.184134')}

In [ ]:
engine.dispose()